# Web3 for Django

**FIRST STEP:**  
Run the code cells in the "Prepare Environment" section.  
Then proceed from here.

```
Variables:
u: Test User
d: Test Dataset
c: Test Contract
```

## Assign Address (& Pre-Fund Account)

In [8]:
# requires create_wallet, fund_wallet and send_ether to be defined
def assign_address_v3(user):
    from web3 import Web3
    import time
    from hexbytes import HexBytes
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    accounts = w3.eth.accounts
    current_user = user
    
    # Create new web3 account
    eth_account = create_wallet()
    
    # Store public key and private key in user model
    current_user.ethereum_public_key = eth_account.address
    current_user.ethereum_private_key = eth_account.privateKey.hex()
    current_user.save()
    
    # Fund wallet
    fund_wallet(recipient = eth_account.address, amount = 100)
    
    # Return user, now with wallet associated
    return current_user

**Test**

In [12]:
assign_address_v3(u)

<User: test@luce.com>

In [23]:
print(u.ethereum_public_key)
print(u.ethereum_private_key)
print("Ether:", w3.fromWei(w3.eth.getBalance(u.ethereum_public_key),'ether'))

0x0e31171C0BEd83361De2e984e5A520C2abb40F22
0x7687047edbf735a5acd777af390f6f84048ef39959f4735fa7e91d8c5c9e6fdb
Ether: 100


## Deploy Contract

In [24]:
def deploy_contract_v3(private_key):
    from solcx import compile_source
    from web3 import Web3
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file: # Adjust file_path for use in Jupyter/Django
        contract_source_code = file.read()
    
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    #Obtain user so we know his address for the 'from' field
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    
    transaction = {
    'from': user.address,
    'gas': 2000000,
    'data': bytecode,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(transaction, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contractAddress = tx_receipt.contractAddress
    
    return contractAddress

**Test**

In [28]:
c = deploy_contract_v3(u.ethereum_private_key)

In [35]:
print("Contract address for Test Dataset:\n", c, sep="")

Contract address for Test Dataset:
0x928e9A49b5fcFaAEf2b8E1C3F3DcBeE12a12E2a6


## Publish Data to Contract

In [37]:
def publish_data_v3(provider_private_key, contract_address, description="Description", license=3, link="void"):
    from web3 import Web3
    
    # Compile Luce contract and obtain interface
    d = compile_and_extract_interface()
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    # Obtain user so we know his address for the 'from' field
    private_key = provider_private_key
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Obtain contract address & instantiate contract
    contract_address = contract_address
    luce = w3.eth.contract(address=contract_address, abi=d['abi'])
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    txn_dict = {
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    luce_txn = luce.functions.publishData(description,link,license).buildTransaction(txn_dict)
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(luce_txn, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    return tx_receipt

**Test**

In [38]:
publish_data_v3(u.ethereum_private_key,c)

## Add Data Requester

In [39]:
def add_requester_v3(requester_private_key, contract_address, license=3, purpose_code=3):
    from web3 import Web3
    
    # Compile Luce contract and obtain interface
    d = compile_and_extract_interface()
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    # Obtain user so we know his address for the 'from' field
    private_key = requester_private_key
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Obtain contract address & instantiate contract
    contract_address = contract_address
    luce = w3.eth.contract(address=contract_address, abi=d['abi'])
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    txn_dict = {
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    license_type = license
    luce_txn = luce.functions.addDataRequester(purpose_code,license_type).buildTransaction(txn_dict)
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(luce_txn, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    return tx_receipt

**Test**

In [41]:
add_requester_v3(u.ethereum_private_key,c)

## Update Contract

In [42]:
def update_contract_v3(provider_private_key, contract_address, description="Updated Description", link="void"):
    from web3 import Web3
    
    # Compile Luce contract and obtain interface
    d = compile_and_extract_interface()
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    # Obtain user so we know his address for the 'from' field
    private_key = provider_private_key
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Obtain contract address & instantiate contract
    contract_address = contract_address
    luce = w3.eth.contract(address=contract_address, abi=d['abi'])
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    txn_dict = {
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    luce_txn = luce.functions.updateData(description,link).buildTransaction(txn_dict)
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(luce_txn, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    return tx_receipt

**Test**

In [43]:
update_contract_v3(u.ethereum_private_key,c)

## Prepare Environment

### Prepare Facuet & Web3 Connection

In [2]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
# Private key (from Ganache interface)
faucet_privateKey   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"
faucet = w3.eth.account.privateKeyToAccount(faucet_privateKey)

### Import Django Context for Testing

In [3]:
# Initialise Django context
import django
django.setup()

# Obtain user model
from django.contrib.auth import get_user_model
User = get_user_model()

# Obtain dataset model
from datastore.models import Dataset

ImproperlyConfigured: Requested setting LOGGING_CONFIG, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [4]:
# Obtain Django test user object
u = User.objects.all()[1]
u

NameError: name 'User' is not defined

In [44]:
# Obtain Django test dataset object
d = Dataset.objects.all()[0]
print(d)
print(d.title)
print(d.file)

Dataset object (10)
Test User Dataset
files/test_cdsbAxG.txt


In [55]:
Dataset.objects.get(id=1).title

'Test Dataset'

### Load Auxillary Functions

In [5]:
# Define function to send ether
import time
def send_ether(amount_in_ether, recipient_address, sender_pkey=faucet.privateKey):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');
    
    # Obtain sender address from private key
    sender_address = w3.eth.account.privateKeyToAccount(sender_pkey).address

    # How many transactions have been made by wallet?
    # This is required and prevents double-spending.
    # Same name but different from nonce in block mining.
    nonce = w3.eth.getTransactionCount(sender_address)
    
    # Specify transcation dictionary
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # IN THIS STEP THE PRIVATE KEY OF THE SENDER IS USED
    # Sign transaction
    def sign_transaction(txn_dict, sender_pkey):
        signed_txn = w3.eth.account.signTransaction(txn_dict, sender_pkey)
        return signed_txn
    signed_txn      = sign_transaction(txn_dict, sender_pkey)
    signed_txn_raw = signed_txn.rawTransaction
    
    
    # Send transaction & store transaction hash
    def send_raw_transaction(raw_transaction):
        txn_hash = w3.eth.sendRawTransaction(raw_transaction)
        return txn_hash
    txn_hash = send_raw_transaction(signed_txn_raw)

    # Check if transaction was added to blockchain 
    # time.sleep(5)     # Not needed on Ganache since our transactions are instantaneous
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    
#     if txn_receipt:
#         return True # The transaction was successful
    return txn_hash

In [6]:
def create_wallet():
    eth_account = w3.eth.account.create()
    return (eth_account)

In [7]:
# Requires send_ether to be defined
def fund_wallet(recipient, amount = 100):
    send_ether(amount,recipient)

In [36]:
def compile_and_extract_interface():
    from solcx import compile_source
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file: # Adjust file_path for use in Jupyter/Django
        contract_source_code = file.read()
        
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Create dictionary with interface
    d = dict()
    d['abi']      = abi
    d['bytecode'] = bytecode
    d['full_interface'] = contract_interface
    return(d)